In [14]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import warnings
warnings.simplefilter('once')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [15]:
import pandas as pd
import numpy as np
import re, time, os
from eunjeon import Mecab

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [16]:
text = ["군 신병교육대에 입소해 훈련을 받고 있는 병들을 편의상 이르는 말.",
        "군대의 병 계급의 첫 번째 단계로, 대한민국의 군사 계급 중 최하위 계급이다. 후술하겠지만 훈련병, 이경, 이방, 이교도 이등병이다.",
       "군대의 병 신분의 계급 중 하나. 흔히 '일병'이라고 부른다",
       "군대의 병 계급의 세 번째 단계. 한국군과 미군에서 가장 숫자가 많은 계급이다.",
       "군대의 병 계급의 최종 단계. 군필자는 전역 후에도 전역 전 계급을 이어가기 때문에, 만기전역을 하면서 영창 등 진급에 영향을 받지 않았다면 행정상 예비군으로서 달고 있는 계급 또한 병장이다."]
df = pd.DataFrame(text, columns=['text'])
df

,text
0,군 신병교육대에 입소해 훈련을 받고 있는 병들을 편의상 이르는 말.
1,"군대의 병 계급의 첫 번째 단계로, 대한민국의 군사 계급 중 최하위 계급이다. 후술..."
2,군대의 병 신분의 계급 중 하나. 흔히 '일병'이라고 부른다
3,군대의 병 계급의 세 번째 단계. 한국군과 미군에서 가장 숫자가 많은 계급이다.
4,군대의 병 계급의 최종 단계. 군필자는 전역 후에도 전역 전 계급을 이어가기 때문에...


In [17]:
mecab = Mecab()

# Stopword Thesaurus Read
def read_stopword(stopword_file):
    with open(stopword_file, 'r', encoding='UTF-8') as f:
        stopwords = f.read().split('\n')
    return list(set(stopwords))

def read_thesaurus(file_name):
    thesaurus = {}
    with open(file_name, 'r', encoding='UTF-8') as f:
        lines = f.readlines()
        
    for line in lines:
        line = line.replace('\n','')
        for tok in line.split(',')[1:]:
            thesaurus[tok] = line.split(',')[0]
    return thesaurus

# 한글만 사용
def cleaning(text):
    pattern = re.compile(r"[^ㄱ-힗]")
    return re.sub(pattern, ' ', str(text))

# 명사 추출기
def tokenizer(text):
    return mecab.nouns(text)

#  Stopword 제거
def drop_stopwords(text, stopword):
    result = []
    for tok in text.split():
        if tok in stopword:
            continue
        else:
            result.append(tok)
    return result

#  시소러스 적용
def change_thesaurus(text, thesaurus):
    result = []
    for tok in text.split():
        if tok in thesaurus.keys():
            result.appeend(thesaurus_dict[tok])
        else:
            result.append(tok)
    return result

# 한글, 명사 추출, 불용어제거, 시소러스 적용
def preprocessing(text, stopword, thesaurus):
    result = []
    text = cleaning(text)
    
    toks = mecab.nouns(text)
#     print(toks)
    for tok in toks:
        if (tok in stopword):
            continue
        else:
            if tok in thesaurus.keys():
                result.append(thesaurus[tok])
            else:
                result.append(tok)
    
    return " ".join(result)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
stopword = read_stopword('stopwords.txt')
stopword

['미군', '계급', '한국군', '군대']

In [19]:
thesaurus = read_thesaurus('thesaurus.txt')
thesaurus

{'이경': '이등병', '이방': '이등병', '이병': '이등병', '이교': '이등병', '병': '병사'}

In [20]:
df['result'] = df['text'].apply(lambda x : preprocessing(x, stopword, thesaurus))
df

,text,result
0,군 신병교육대에 입소해 훈련을 받고 있는 병들을 편의상 이르는 말.,군 신병교육대 입소 훈련 병사 말
1,"군대의 병 계급의 첫 번째 단계로, 대한민국의 군사 계급 중 최하위 계급이다. 후술...",병사 번 단계 대한민국 군사 중 하위 후술 훈련병 이등병 방 교도 이등병
2,군대의 병 신분의 계급 중 하나. 흔히 '일병'이라고 부른다,병사 신분 중 하나 일병
3,군대의 병 계급의 세 번째 단계. 한국군과 미군에서 가장 숫자가 많은 계급이다.,병사 번 단계 국군 숫자
4,군대의 병 계급의 최종 단계. 군필자는 전역 후에도 전역 전 계급을 이어가기 때문에...,병사 최종 단계 군필 전역 후 전역 전 때문 전역 영창 등 진급 영향 행정 예비군 병장
